# Serving a PyTorch Model as a REST Endpoint with TorchServe and SageMaker

We will deploy our BERT PyTorch Model as a REST Endpoint on SageMaker using TorchServe https://github.com/pytorch/serve/

TorchServe can be used for many types of inference in production settings. It provides an easy-to-use command line interface and utilizes REST based APIs handle state prediction requests.

<img src="./img/torchserve.png" width="90%">
  
More information on how to deploy Huggingface Transformers with TorchServe:
* https://github.com/pytorch/serve/tree/master/examples/Huggingface_Transformers
* https://medium.com/analytics-vidhya/deploy-huggingface-s-bert-to-production-with-pytorch-serve-27b068026d18 

In [1]:
import boto3
import sagemaker
import pandas as pd

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
account_id = boto3.client('sts').get_caller_identity().get('Account')

sm = boto3.Session().client(service_name='sagemaker', region_name=region)

# PRE-REQUISITE: 

## You need to have succesfully run the notebooks in the `TRAINING` section and converted your TF model into PyTorch before you continue with this notebook. 

In [2]:
%store -r training_job_name

In [3]:
try:
    training_job_name
    print('[OK]')
except NameError:
    print('+++++++++++++++++++++++++++++++')
    print('[ERROR] Please run the notebooks in the previous TRAIN section before you continue.')
    print('+++++++++++++++++++++++++++++++')

[OK]


In [4]:
print(training_job_name)

tensorflow-training-2020-09-26-18-44-25-975


In [5]:
%store -r transformer_pytorch_model_s3_uri

In [6]:
try:
    transformer_pytorch_model_s3_uri
    print('[OK]')
except NameError:
    print('+++++++++++++++++++++++++++++++')
    print('[ERROR] Please run the notebooks in the previous TRAIN section before you continue.')
    print('+++++++++++++++++++++++++++++++')

[OK]


In [7]:
print(transformer_pytorch_model_s3_uri)

s3://sagemaker-us-west-2-085964654406/models/transformer-pytorch/


# Copy the Transformer PyTorch Model from S3 to Local

In [8]:
local_model_dir = './models/transformers/pytorch/'

In [9]:
!aws s3 cp --recursive $transformer_pytorch_model_s3_uri $local_model_dir

download: s3://sagemaker-us-west-2-085964654406/models/transformer-pytorch/config.json to models/transformers/pytorch/config.json
download: s3://sagemaker-us-west-2-085964654406/models/transformer-pytorch/pytorch_model.bin to models/transformers/pytorch/pytorch_model.bin


# Retrieve Transformer PyTorch Model Name (.bin) Created During Training

In [10]:
%store -r transformer_pytorch_model_name

In [11]:
try:
    transformer_pytorch_model_name
    print('[OK]')
except NameError:
    print('+++++++++++++++++++++++++++++++')
    print('[ERROR] Please run the notebooks in the previous TRAIN section before you continue.')
    print('+++++++++++++++++++++++++++++++')

[OK]


In [12]:
print(transformer_pytorch_model_name)

pytorch_model.bin


# Create TorchServe Model Archive File (.mar)

https://github.com/pytorch/serve/blob/master/model-archiver/README.md

A key feature of TorchServe is the ability to package all model artifacts into a single model archive file. It is a separate command line interface (CLI), torch-model-archiver, that can take model checkpoints or model definition file with state_dict, and package them into a .mar file. This file can then be redistributed and served by anyone using TorchServe. It takes in the following model artifacts: a model checkpoint file in case of torchscript or a model definition file and a state_dict file in case of eager mode, and other optional assets that may be required to serve the model. The CLI creates a .mar file that TorchServe's server CLI uses to serve the models. 

We need to pass the the following:
* `--handler`:  Python code to adapt the `review_body` to BERT tokens (request handler) as well as the `star_rating` response of 1-5 (response handler)
* `config.json`:  used by the Huggingface transformers library when we saved the model in a previous notebook.  In 
* `setup_config.json`:  BERT-specific `setup_config.json` that defines the `max seq length`, `number of output classes` (1-5), etc.
* `Seq_classification_artifacts/index_to_name.json`:  BERT-specific mapping of response index (0-4) to class name (1-5 star rating) for our output classes

In [13]:
!mkdir -p ./model_store

In [14]:
!torch-model-archiver -f \
    --model-name model \
    --export-path ./model_store/ \
    --version 1.0 \
    --serialized-file $local_model_dir/$transformer_pytorch_model_name \
    --handler ./src_torchserve/Transformer_handler_generalized.py \
    --extra-files "./models/transformers/pytorch/config.json,./src_torchserve/setup_config.json,./src_torchserve/Seq_classification_artifacts/index_to_name.json"

In [15]:
!ls -al ./model_store/

total 241224
drwxrwxr-x  2 ec2-user ec2-user      4096 Sep 26 20:10 .
drwxrwxr-x 13 ec2-user ec2-user      4096 Sep 26 20:10 ..
-rw-rw-r--  1 ec2-user ec2-user 247002150 Sep 26 20:10 model.mar


# Start TorchServe locally to serve the model

After you archive and store the model, use the torchserve command to serve the model.

# Prepare the Model for SageMaker Deployment

To deploy the model to a SageMaker REST endpoint, we need to upload our .mar file to S3 and build a TorchServe model container. 

In [16]:
!unzip -o ./model_store/model.mar

Archive:  ./model_store/model.mar
  inflating: pytorch_model.bin       
  inflating: index_to_name.json      
  inflating: setup_config.json       
  inflating: Transformer_handler_generalized.py  
  inflating: config.json             
  inflating: MAR-INF/MANIFEST.json   


# Upload TorchServe Model Archive File to S3

In [17]:
torchserve_mar = 'model.mar'

# Tar the `.mar` Archive File as `model.tar.gz` and Upload to S3
Per TorchServe convention, the `.mar` file must be under ./model_store/ in the `.tar` archive

In [18]:
!mkdir -p ./tmp/
!tar -cvzf ./tmp/model.tar.gz \
    ./model_store/$torchserve_mar

./model_store/model.mar


In [19]:
tmp_torchserve_model_name = 'reviews-distilbert-pytorch'

print(tmp_torchserve_model_name)

reviews-distilbert-pytorch


In [20]:
tmp_torchserve_tar_s3_uri = 's3://{}/models/torchserve/model.tar.gz'.format(bucket, tmp_torchserve_model_name)

print(tmp_torchserve_tar_s3_uri)

s3://sagemaker-us-west-2-085964654406/models/torchserve/model.tar.gz


# Upload `model.tar.gz` to S3

In [21]:
!aws s3 cp ./tmp/model.tar.gz $tmp_torchserve_tar_s3_uri

upload: tmp/model.tar.gz to s3://sagemaker-us-west-2-085964654406/models/torchserve/model.tar.gz


In [22]:
print(tmp_torchserve_tar_s3_uri)

s3://sagemaker-us-west-2-085964654406/models/torchserve/model.tar.gz


In [23]:
!aws s3 ls $tmp_torchserve_tar_s3_uri

2020-09-26 20:10:34  247041274 model.tar.gz


# Build a TorchServe Docker Image

In [24]:
!pygmentize ./docker/Dockerfile

# syntax = docker/dockerfile:experimental
#
# This file can build images for cpu and gpu env. By default it builds image for CPU.
# Use following option to build image for cuda/GPU: --build-arg BASE_IMAGE=nvidia/cuda:10.1-cudnn7-runtime-ubuntu18.04
# Here is complete command for GPU/cuda - 
# $ DOCKER_BUILDKIT=1 docker build --file Dockerfile --build-arg BASE_IMAGE=nvidia/cuda:10.1-cudnn7-runtime-ubuntu18.04 -t torchserve:latest .
#
# Following comments have been shamelessly copied from https://github.com/pytorch/pytorch/blob/master/Dockerfile
# 
# NOTE: To build this you will need a docker version > 18.06 with
#       experimental enabled and DOCKER_BUILDKIT=1
#
#       If you do not use buildkit you are not going to have a good time
#
#       For reference: 
#           https://docs.docker.com/develop/develop-images/build_enhancements/


ARG BASE_IMAGE=ubuntu:18.04

FROM ${BASE_IMAGE} AS compile-image

ENV PYTHONUNBUFFERED TRUE

# RUN --mount=type=cache,id=apt-dev,target=/var/cache/a

In [25]:
docker_repo = 'torchserve'
docker_tag = 'torch-1.5.0-1.0.0'

image_uri = f'{account_id}.dkr.ecr.{region}.amazonaws.com/{docker_repo}:{docker_tag}'

In [26]:
!docker build -t $docker_repo:$docker_tag -f ./docker/Dockerfile ./docker

Sending build context to Docker daemon  6.656kB
Step 1/28 : ARG BASE_IMAGE=ubuntu:18.04
Step 2/28 : FROM ${BASE_IMAGE} AS compile-image
18.04: Pulling from library/ubuntu

57c49d0f: Pulling fs layer 
40447d26: Pulling fs layer 
Digest: sha256:646942475da61b4ce9cc5b3fadb42642ea90e5d0de46111458e100ff2c7031e6
Status: Downloaded newer image for ubuntu:18.04
 ---> 56def654ec22
Step 3/28 : ENV PYTHONUNBUFFERED TRUE
 ---> Running in 3d63d6379a88
Removing intermediate container 3d63d6379a88
 ---> c6c670353e84
Step 4/28 : RUN apt-get update &&     DEBIAN_FRONTEND=noninteractive apt-get install --no-install-recommends -y     ca-certificates     g++     python3-dev     python3-distutils     python3-venv     openjdk-11-jre-headless     curl     && rm -rf /var/lib/apt/lists/*     && cd /tmp     && curl -O https://bootstrap.pypa.io/get-pip.py     && python3 get-pip.py
 ---> Running in 58c8bc8f6393
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 http://archive.ubuntu

Get:20 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libx11-data all 2:1.6.4-3ubuntu0.3 [114 kB]
Get:21 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libx11-6 amd64 2:1.6.4-3ubuntu0.3 [571 kB]
Get:22 http://archive.ubuntu.com/ubuntu bionic/main amd64 libxext6 amd64 2:1.3.3-1 [29.4 kB]
Get:23 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libjpeg-turbo8 amd64 1.5.2-0ubuntu5.18.04.4 [110 kB]
Get:24 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 openssl amd64 1.1.1-1ubuntu2.1~18.04.6 [614 kB]
Get:25 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 ca-certificates all 20190110~18.04.1 [146 kB]
Get:26 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libdbus-1-3 amd64 1.12.2-1ubuntu1.2 [175 kB]
Get:27 http://archive.ubuntu.com/ubuntu bionic/main amd64 ucf all 3.0038 [50.5 kB]
Get:28 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libkrb5support0 amd64 1.16-2ubuntu0.1 [30.9 kB]
Get:29 http://archive.ubuntu.com/ub

Get:99 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 gcc-7 amd64 7.5.0-3ubuntu1~18.04 [9381 kB]
Get:100 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 gcc amd64 4:7.4.0-1ubuntu2.3 [5184 B]
Get:101 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libc-dev-bin amd64 2.27-3ubuntu1.2 [71.8 kB]
Get:102 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 linux-libc-dev amd64 4.15.0-118.119 [994 kB]
Get:103 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libc6-dev amd64 2.27-3ubuntu1.2 [2585 kB]
Get:104 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libstdc++-7-dev amd64 7.5.0-3ubuntu1~18.04 [1471 kB]
Get:105 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 g++-7 amd64 7.5.0-3ubuntu1~18.04 [9697 kB]
Get:106 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 g++ amd64 4:7.4.0-1ubuntu2.3 [1568 B]
Get:107 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libexpat1-dev amd64 2.2.5-3ubuntu0.2 [122 kB]
Get:1

Selecting previously unselected package libpsl5:amd64.
Preparing to unpack .../12-libpsl5_0.19.1-5build1_amd64.deb ...
Unpacking libpsl5:amd64 (0.19.1-5build1) ...
Selecting previously unselected package binutils-common:amd64.
Preparing to unpack .../13-binutils-common_2.30-21ubuntu1~18.04.4_amd64.deb ...
Unpacking binutils-common:amd64 (2.30-21ubuntu1~18.04.4) ...
Selecting previously unselected package libbinutils:amd64.
Preparing to unpack .../14-libbinutils_2.30-21ubuntu1~18.04.4_amd64.deb ...
Unpacking libbinutils:amd64 (2.30-21ubuntu1~18.04.4) ...
Selecting previously unselected package binutils-x86-64-linux-gnu.
Preparing to unpack .../15-binutils-x86-64-linux-gnu_2.30-21ubuntu1~18.04.4_amd64.deb ...
Unpacking binutils-x86-64-linux-gnu (2.30-21ubuntu1~18.04.4) ...
Selecting previously unselected package binutils.
Preparing to unpack .../16-binutils_2.30-21ubuntu1~18.04.4_amd64.deb ...
Unpacking binutils (2.30-21ubuntu1~18.04.4) ...
Selecting previously unselected package java-co

Selecting previously unselected package libldap-2.4-2:amd64.
Preparing to unpack .../57-libldap-2.4-2_2.4.45+dfsg-1ubuntu1.6_amd64.deb ...
Unpacking libldap-2.4-2:amd64 (2.4.45+dfsg-1ubuntu1.6) ...
Selecting previously unselected package libnghttp2-14:amd64.
Preparing to unpack .../58-libnghttp2-14_1.30.0-1ubuntu1_amd64.deb ...
Unpacking libnghttp2-14:amd64 (1.30.0-1ubuntu1) ...
Selecting previously unselected package librtmp1:amd64.
Preparing to unpack .../59-librtmp1_2.4+20151223.gitfa8646d.1-1_amd64.deb ...
Unpacking librtmp1:amd64 (2.4+20151223.gitfa8646d.1-1) ...
Selecting previously unselected package libcurl4:amd64.
Preparing to unpack .../60-libcurl4_7.58.0-2ubuntu3.10_amd64.deb ...
Unpacking libcurl4:amd64 (7.58.0-2ubuntu3.10) ...
Selecting previously unselected package curl.
Preparing to unpack .../61-curl_7.58.0-2ubuntu3.10_amd64.deb ...
Unpacking curl (7.58.0-2ubuntu3.10) ...
Selecting previously unselected package python3-lib2to3.
Preparing to unpack .../62-python3-lib2to3

...

  Running setup.py install for future: started
    Running setup.py install for future: finished with status 'done'
  Running setup.py install for psutil: started
    Running setup.py install for psutil: finished with status 'done'
Removing intermediate container a53492172091
 ---> 042255082881
Step 11/28 : RUN pip install --no-cache-dir transformers==2.8.0
 ---> Running in f8ec71c08239
  Running setup.py install for sacremoses: started
    Running setup.py install for sacremoses: finished with status 'done'
Removing intermediate container f8ec71c08239
 ---> bf09664ecbc8
Step 12/28 : RUN pip list
 ---> Running in 5498a289fb7a
DEPRECATION: The default format will switch to columns in the future. You can use --format=(legacy|columns) (or define a format=(legacy|columns) in your pip.conf under the [list] section) to disable this warning.
boto3 (1.15.6)
botocore (1.18.6)
certifi (2020.6.20)
chardet (3.0.4)
click (7.1.2)
dataclasses (0.7)
enum-compat (0.0.3)
filelock (3.0.12)
future (0.18.2

Get:11 http://archive.ubuntu.com/ubuntu bionic/multiverse amd64 Packages [186 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic/restricted amd64 Packages [13.5 kB]
Get:13 http://archive.ubuntu.com/ubuntu bionic-updates/restricted amd64 Packages [146 kB]
Get:14 http://archive.ubuntu.com/ubuntu bionic-updates/multiverse amd64 Packages [33.9 kB]
Get:15 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 Packages [1434 kB]
Get:16 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 Packages [1406 kB]
Get:17 http://archive.ubuntu.com/ubuntu bionic-backports/universe amd64 Packages [8432 B]
Get:18 http://archive.ubuntu.com/ubuntu bionic-backports/main amd64 Packages [8286 B]
Fetched 18.6 MB in 4s (5028 kB/s)
Reading package lists...
Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  ca-certificates ca-certificates-java fontconfig-config fonts-dejavu-core
  java-common libasound2 libasound2

Get:55 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 ca-certificates-java all 20180516ubuntu1~18.04.1 [12.2 kB]
debconf: delaying package configuration, since apt-utils is not installed
Fetched 50.3 MB in 6s (7893 kB/s)
Selecting previously unselected package libssl1.1:amd64.
(Reading database ... 4045 files and directories currently installed.)
Preparing to unpack .../libssl1.1_1.1.1-1ubuntu2.1~18.04.6_amd64.deb ...
Unpacking libssl1.1:amd64 (1.1.1-1ubuntu2.1~18.04.6) ...
Selecting previously unselected package libpython3.6-minimal:amd64.
Preparing to unpack .../libpython3.6-minimal_3.6.9-1~18.04ubuntu1.1_amd64.deb ...
Unpacking libpython3.6-minimal:amd64 (3.6.9-1~18.04ubuntu1.1) ...
Selecting previously unselected package libexpat1:amd64.
Preparing to unpack .../libexpat1_2.2.5-3ubuntu0.2_amd64.deb ...
Unpacking libexpat1:amd64 (2.2.5-3ubuntu0.2) ...
Selecting previously unselected package python3.6-minimal.
Preparing to unpack .../python3.6-minimal_3.6.9-1~18.04ubuntu1.

Selecting previously unselected package fontconfig-config.
Preparing to unpack .../21-fontconfig-config_2.12.6-0ubuntu2_all.deb ...
Unpacking fontconfig-config (2.12.6-0ubuntu2) ...
Selecting previously unselected package libfontconfig1:amd64.
Preparing to unpack .../22-libfontconfig1_2.12.6-0ubuntu2_amd64.deb ...
Unpacking libfontconfig1:amd64 (2.12.6-0ubuntu2) ...
Selecting previously unselected package libnspr4:amd64.
Preparing to unpack .../23-libnspr4_2%3a4.18-1ubuntu1_amd64.deb ...
Unpacking libnspr4:amd64 (2:4.18-1ubuntu1) ...
Selecting previously unselected package libnss3:amd64.
Preparing to unpack .../24-libnss3_2%3a3.35-2ubuntu2.12_amd64.deb ...
Unpacking libnss3:amd64 (2:3.35-2ubuntu2.12) ...
Selecting previously unselected package libasound2-data.
Preparing to unpack .../25-libasound2-data_1.1.3-5ubuntu0.5_all.deb ...
Unpacking libasound2-data (1.1.3-5ubuntu0.5) ...
Selecting previously unselected package libasound2:amd64.
Preparing to unpack .../26-libasound2_1.1.3-5ubunt

Adding debian:SwissSign_Silver_CA_-_G2.pem
Adding debian:VeriSign_Class_3_Public_Primary_Certification_Authority_-_G4.pem
Adding debian:SSL.com_Root_Certification_Authority_ECC.pem
Adding debian:Autoridad_de_Certificacion_Firmaprofesional_CIF_A62634068.pem
Adding debian:Entrust_Root_Certification_Authority_-_G2.pem
Adding debian:GeoTrust_Primary_Certification_Authority.pem
Adding debian:DST_Root_CA_X3.pem
Adding debian:COMODO_ECC_Certification_Authority.pem
Adding debian:GlobalSign_Root_CA_-_R6.pem
Adding debian:VeriSign_Universal_Root_Certification_Authority.pem
Adding debian:TWCA_Root_Certification_Authority.pem
Adding debian:Hellenic_Academic_and_Research_Institutions_ECC_RootCA_2015.pem
Adding debian:GeoTrust_Global_CA.pem
Adding debian:Cybertrust_Global_Root.pem
Adding debian:AffirmTrust_Networking.pem
Adding debian:COMODO_Certification_Authority.pem
Adding debian:Deutsche_Telekom_Root_CA_2.pem
Adding debian:thawte_Primary_Root_CA_-_G2.pem
Adding debian:ISRG_Root_X1.pem
Adding deb

# Check the Docker Image
If the image did not build properly, re-run the cell above.

In [27]:
!docker inspect $docker_repo:$docker_tag

[
    {
        "Id": "sha256:e22ad0e804727d7b0d3f789729cd3c302ce36b66e41daad8bd62f454fdaef5e1",
        "RepoTags": [
            "torchserve:torch-1.5.0-1.0.0"
        ],
        "RepoDigests": [],
        "Parent": "sha256:3a9fcba995ce8b49d128985d6e68c47f054d8396af074c8bc63023c8dc190e1e",
        "Comment": "",
        "Created": "2020-09-26T20:14:00.214898284Z",
        "Container": "17d23e5e011d03dce014bd913220cc7328c1dddb7ea66908723bfd592f884457",
        "ContainerConfig": {
            "Hostname": "17d23e5e011d",
            "Domainname": "",
            "User": "model-server",
            "AttachStdin": false,
            "AttachStdout": false,
            "AttachStderr": false,
            "ExposedPorts": {
                "8080/tcp": {},
                "8081/tcp": {}
            },
            "Tty": false,
            "OpenStdin": false,
            "StdinOnce": false,
            "Env": [
                "PATH=/home/venv/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/b

# Push the Image to a Private Docker Repo (Amazon ECR)

In [28]:
import boto3
account_id = boto3.client('sts').get_caller_identity().get('Account')
region = boto3.session.Session().region_name

image_uri = '{}.dkr.ecr.{}.amazonaws.com/{}:{}'.format(account_id, region, docker_repo, docker_tag)
print(image_uri)

085964654406.dkr.ecr.us-west-2.amazonaws.com/torchserve:torch-1.5.0-1.0.0


In [29]:
!$(aws ecr get-login --region $region --registry-ids $account_id --no-include-email)

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


### Ignore the `RepositoryNotFoundException` Error Below

In [30]:
!aws ecr describe-repositories --repository-names $docker_repo || aws ecr create-repository --repository-name $docker_repo


An error occurred (RepositoryNotFoundException) when calling the DescribeRepositories operation: The repository with name 'torchserve' does not exist in the registry with id '085964654406'
{
    "repository": {
        "repositoryArn": "arn:aws:ecr:us-west-2:085964654406:repository/torchserve",
        "registryId": "085964654406",
        "repositoryName": "torchserve",
        "repositoryUri": "085964654406.dkr.ecr.us-west-2.amazonaws.com/torchserve",
        "createdAt": 1601151245.0,
        "imageTagMutability": "MUTABLE",
        "imageScanningConfiguration": {
            "scanOnPush": false
        },
        "encryptionConfiguration": {
            "encryptionType": "AES256"
        }
    }
}


In [31]:
!docker tag $docker_repo:$docker_tag $image_uri

In [32]:
!docker push $image_uri

The push refers to repository [085964654406.dkr.ecr.us-west-2.amazonaws.com/torchserve]

c40539bf: Preparing 
8ffaa37a: Preparing 
4bcb182b: Preparing 
e295592a: Preparing 
0fd8286d: Preparing 
03d13725: Preparing 
21f132b8: Preparing 
4df0ad6c: Preparing 
df553184: Preparing 
03d13725: Pushed   1.595GB/1.588GBtorch-1.5.0-1.0.0: digest: sha256:b3960d9acead9e7481bbe41b791da9cc1a57d3c59a16ed8ce5be27abf9ef9b1d size: 2405


## Create SageMaker Endpoint and Deploy TorchServe Model Container

In [33]:
import time
timestamp = int(time.time())

pytorch_model_name = '{}-{}-{}'.format(training_job_name, 'pt', timestamp)

print(pytorch_model_name)

tensorflow-training-2020-09-26-18-44-25-975-pt-1601151312


In [34]:
from sagemaker.model import Model
from sagemaker.predictor import Predictor

pytorch_model = Model(image_uri=image_uri,
                      model_data=tmp_torchserve_tar_s3_uri,                       
                      role=role,
                      predictor_cls=Predictor,
                      name=pytorch_model_name)

In [35]:
%store pytorch_model_name

Stored 'pytorch_model_name' (str)


In [36]:
import time

pytorch_endpoint_name = '{}-{}-{}'.format(training_job_name, 'pt', timestamp)

In [37]:
print(pytorch_endpoint_name)

predictor = pytorch_model.deploy(instance_type='ml.m5.large',
                                 initial_instance_count=1,
                                 endpoint_name=pytorch_endpoint_name,
                                 wait=False)

tensorflow-training-2020-09-26-18-44-25-975-pt-1601151312


In [38]:
from IPython.core.display import display, HTML

display(HTML('<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/endpoints/{}">REST Endpoint</a></b>'.format(region, pytorch_endpoint_name)))


# _Wait Until the Endpoint is Deployed_

In [39]:
%%time

waiter = sm.get_waiter('endpoint_in_service')
waiter.wait(EndpointName=pytorch_endpoint_name)

CPU times: user 161 ms, sys: 7.22 ms, total: 169 ms
Wall time: 7min 31s


# _Wait Until the ^^ Endpoint ^^ is Deployed_

### Waiting for the Endpoint to be ready to Serve Predictions

In [40]:
import time

time.sleep(30)

# Predict the `star_rating` with `review_body` Samples from our TSV's

In [41]:
import csv

df_reviews = pd.read_csv('./data/amazon_reviews_us_Digital_Software_v1_00.tsv.gz', 
                         delimiter='\t', 
                         quoting=csv.QUOTE_NONE,
                         compression='gzip')
df_sample_reviews = df_reviews[['review_body', 'star_rating']].sample(n=50)
df_sample_reviews = df_sample_reviews.reset_index()
df_sample_reviews.shape

(50, 3)

In [42]:
import pandas as pd

def predict(review_body):
    return predictor.predict(review_body).decode('utf-8')

df_sample_reviews['predicted_class'] = df_sample_reviews['review_body'].map(predict)
df_sample_reviews.head(5)

,index,review_body,star_rating,predicted_class
0,52693,It blocks viruses successfully and meets my ba...,5,3
1,88774,"I have used office for years, since even befor...",2,5
2,61481,I use word perfect at work and this is perfect...,5,3
3,95886,I am happy with this product. I was also pleas...,4,4
4,93937,There are many different ideas on the best way...,1,3


# Predict the `star_rating` with Ad Hoc `review_body` Samples

In [43]:
predicted_classes = predictor.predict('This is great!')

print(predicted_classes.decode('utf-8'))

5


In [44]:
list(predictor._model_names)

['tensorflow-training-2020-09-26-18-44-25-975-pt-1601151312']

# Save for Next Notebook(s)

In [45]:
%store pytorch_endpoint_name

Stored 'pytorch_endpoint_name' (str)


In [46]:
%store

Stored variables and their in-db values:
auto_ml_job_name                                      -> 'automl-dm-26-16-00-25'
autopilot_endpoint_name                               -> 'automl-dm-ep-26-16-21-49'
autopilot_train_s3_uri                                -> 's3://sagemaker-us-west-2-085964654406/data/amazon
balance_dataset                                       -> True
experiment_name                                       -> 'Amazon-Customer-Reviews-BERT-Experiment-160114585
ingest_create_athena_db_passed                        -> True
ingest_create_athena_table_parquet_passed             -> True
ingest_create_athena_table_tsv_passed                 -> True
max_seq_length                                        -> 64
prepare_trial_component_name                          -> 'TrialComponent-2020-09-26-184417-oahc'
processed_test_data_s3_uri                            -> 's3://sagemaker-us-west-2-085964654406/sagemaker-s
processed_train_data_s3_uri                           -> 's3://sa

## Delete Endpoint
To save money, we should delete the endpoint.

In [47]:
# sm.delete_endpoint(
#     EndpointName=pytorch_endpoint_name
# )

In [48]:
%%javascript
Jupyter.notebook.save_checkpoint();
Jupyter.notebook.session.delete();